In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/SI630/

/content/drive/My Drive/SI630


In [4]:
!pwd

/content/drive/My Drive/SI630


In [5]:
import torch

In [6]:
print ('Available devices ', torch.cuda.device_count())

Available devices  1


In [7]:
import json
import pandas as pd
import random

## Create the dataframe for train_set, dev_set and test_set and save them to csv files

In [8]:
with open('song-lyrics.train.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
train_human = []
for json_str in json_list:
    result = json.loads(json_str)
    train_human.append(result['lyrics'])

In [ ]:
len(train_human)

677272

In [9]:
train_human = random.sample(train_human, 10000)

In [10]:
with open('lyrics.machine-gen.train.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
train_machine = []
for json_str in json_list:
    result = json.loads(json_str)
    train_machine.append(result['lyrics'])

In [11]:
len(train_machine)

10000

In [12]:
train_label = [1]*10000 + [0]*10000

In [13]:
train_lyrics = train_human + train_machine

In [14]:
train_set = pd.DataFrame({'text':train_lyrics, 'label':train_label})

In [33]:
train_set.to_csv(r'train_set.csv', index = False, header=True)

In [16]:
with open('song-lyrics.dev.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
dev_human = []
for json_str in json_list:
    result = json.loads(json_str)
    dev_human.append(result['lyrics'])

In [17]:
len(dev_human)

84700

In [18]:
with open('lyrics.machine-gen.dev.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
dev_machine = []
for json_str in json_list:
    result = json.loads(json_str)
    dev_machine.append(result['lyrics'])

In [19]:
len(dev_machine)

11124

In [20]:
dev_human = random.sample(dev_human, 5000)
dev_machine = random.sample(dev_machine, 5000)

In [21]:
dev_label = [1]*5000 + [0]*5000

In [22]:
dev_lyrics = dev_human + dev_machine

In [23]:
dev_set = pd.DataFrame({'text':dev_lyrics, 'label':dev_label})

In [24]:
dev_set.to_csv(r'dev_set.csv', index = False, header=True) 

In [25]:
with open('song-lyrics.test.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
test_human = []
for json_str in json_list:
    result = json.loads(json_str)
    test_human.append(result['lyrics'])

In [26]:
len(test_human)

84484

In [27]:
with open('lyrics.machine-gen.test.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
test_machine = []
for json_str in json_list:
    result = json.loads(json_str)
    test_machine.append(result['lyrics'])

In [28]:
len(test_machine)

11124

In [29]:
test_human = random.sample(test_human, 5000)
test_machine = random.sample(test_machine, 5000)

In [30]:
test_label =  [1]*5000 + [0]*5000
test_lyrics = test_human + test_machine

In [31]:
test_set = pd.DataFrame({'text':test_lyrics, 'label':test_label})

In [32]:
test_set.to_csv(r'test_set.csv', index = False, header=True)

In [42]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 20.4MB/s 
     |████████████████████████████████| 204kB 49.4MB/s 
     |████████████████████████████████| 3.3MB 52.5MB/s 
     |████████████████████████████████| 2.1MB 48.6MB/s 
     |████████████████████████████████| 122kB 59.0MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 2.1MB 48.3MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 8.2MB 43.4MB/s 
     |████████████████████████████████| 1.2MB 51.7MB/s 
     |████████████████████████████████| 112kB 54.3MB/s 
     |████████████████████████████████| 245kB 54.8MB/s 
     |████████████████████████████████| 901kB 50.9MB/s 
     |████████████████████████████████| 163kB 55.4MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 133kB 53.6MB/s 
     |████████████████████████████████| 112kB 61.9MB/s 
     |████████████████████████████████| 81kB 10.3MB

In [ ]:
#!pip install -U --user tqdm

In [ ]:
#!pip install tqdm

### Read the dataframes 

In [ ]:
train_set = pd.read_csv('train_set.csv')
dev_set = pd.read_csv('dev_set.csv')
test_set = pd.read_csv('test_set.csv')

In [ ]:
train_set = train_set.drop(columns=['Unnamed: 0'])
dev_set = dev_set.drop(columns=['Unnamed: 0'])
test_set = test_set.drop(columns=['Unnamed: 0'])

In [ ]:
train_set = train_set.rename(columns={"lyrics": "text"})
dev_set = dev_set.rename(columns={"lyrics": "text"})
test_set = test_set.rename(columns={"lyrics": "text"})

In [36]:
train_set.head(1)

,text,label
0,Let's run naked through these city streets\nWe...,1


In [37]:
dev_set.head(1)

,text,label
0,If you ask me\nWhat our love's all about\nI wo...,1


In [38]:
test_set.head(1)

,text,label
0,Alice in chains she's had on all her life\nShe...,1


In [39]:
train_set.shape

(20000, 2)

In [40]:
dev_set.shape

(10000, 2)

### Train the model (distilbert-base-cased model)

In [43]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging



In [ ]:
#from tqdm import tqdm

In [44]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [50]:
model_args = ClassificationArgs(num_train_epochs=5)
model_args.labels_list = [1, 0]

In [51]:
model = ClassificationModel('distilbert', 'distilbert-base-cased', args=model_args, use_cuda=True)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

In [ ]:
#model.overwriteOutput = True

In [52]:
model.train_model(train_set)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:449: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_distilbert_128_2_2


INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.


(12500, 0.17414161686076404)

In [53]:
result, model_outputs, wrong_predictions = model.eval_model(dev_set)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_distilbert_128_2_2


INFO:simpletransformers.classification.classification_model:{'mcc': 0.7696872294240474, 'tp': 4677, 'tn': 4150, 'fp': 850, 'fn': 323, 'auroc': 0.9592433600000001, 'auprc': 0.9614041946072966, 'eval_loss': 0.859219140016484}


In [54]:
from sklearn.metrics import f1_score

In [55]:
predictions_dev, raw_outputs_dev = model.predict(dev_lyrics)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


In [65]:
F1_dev = f1_score(dev_label, predictions_dev)

In [66]:
F1_dev

0.876174390372638

In [58]:
result_test, model_outputs_test, wrong_predictions_test = model.eval_model(test_set)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_distilbert_128_2_2


INFO:simpletransformers.classification.classification_model:{'mcc': 0.76671959873509, 'tp': 4671, 'tn': 4141, 'fp': 859, 'fn': 329, 'auroc': 0.96040828, 'auprc': 0.9628128567136667, 'eval_loss': 0.8673906275586923}


In [59]:
predictions_test, raw_outputs_test = model.predict(test_lyrics)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


In [64]:
F1_test = f1_score(test_label, predictions_test)
F1_test

0.8745512143611405

In [102]:
result_train, model_outputs_train, wrong_predictions_train = model.eval_model(train_set)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_distilbert_128_2_2


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9961054238087991, 'tp': 9997, 'tn': 9964, 'fp': 36, 'fn': 3, 'auroc': 0.9999604400000001, 'auprc': 0.9999689941413655, 'eval_loss': 0.009878700434791972}


In [104]:
predictions_train, raw_outputs_train = model.predict(train_lyrics)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


In [105]:
F1_train = f1_score(train_label, predictions_train)
F1_train

0.9980467771823508

### Evaluate the accuracy of 550 lyrics by this model

> Indented block



In [ ]:
#from collections import Counter

In [61]:
with open('lyrics_550_new.json') as json_file:
    lyrics_550 = json.load(json_file)

In [62]:
lyrics_550_list = list(lyrics_550.values())

In [94]:
label_550 = [0]*550

In [95]:
lyrics550_set = pd.DataFrame({'text':lyrics_550_list, 'label':label_550})

In [96]:
result_550, model_outputs_550, wrong_predictions_550 = model.eval_model(lyrics550_set)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_distilbert_128_2_2


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:800: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)
INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'tp': 515, 'tn': 0, 'fp': 0, 'fn': 35, 'auroc': nan, 'auprc': 1.0, 'eval_loss': 0.6020501784572896}


In [97]:
predictions_550, raw_outputs_550 = model.predict(lyrics_550_list)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


In [98]:
F1_550 = f1_score(label_550, predictions_550, average='binary')
F1_550

0.0

Something wrong with the F1 score creation, so I calculated by hand




In [100]:
precision = 515/(515+0)
recall = 515/(515+35)

In [101]:
f1_550 = 2*precision*recall/(precision+recall)
f1_550

0.9671361502347418